In [2]:
pip install requests

In [3]:
import requests

# URL of the notebook raw file
url = "https://raw.githubusercontent.com/adnanarnaout/machine-learning-dse-i210-final-project-nyc-car-accident-severity/main/notebooks/Establish-baseline-logistic-regression-model-AA.ipynb"

# Make a GET request to fetch the raw content of the notebook
r = requests.get(url)
if r.status_code == 200:
    # Saving the notebook
    with open('local_notebook.ipynb', 'wb') as f:
        f.write(r.content)
    print("Notebook downloaded successfully.")
else:
    print("Failed to download the notebook. Status code:", r.status_code)


Notebook downloaded successfully.


In [4]:
%run './local_notebook.ipynb'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057417 entries, 0 to 1057416
Data columns (total 15 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   CRASH DATE                     1057417 non-null  object 
 1   CRASH TIME                     1057417 non-null  object 
 2   LATITUDE                       1057417 non-null  float64
 3   LONGITUDE                      1057417 non-null  float64
 4   CONTRIBUTING FACTOR VEHICLE 1  1057417 non-null  object 
 5   CONTRIBUTING FACTOR VEHICLE 2  1057417 non-null  object 
 6   CONTRIBUTING FACTOR VEHICLE 3  1057417 non-null  object 
 7   CONTRIBUTING FACTOR VEHICLE 4  1057417 non-null  object 
 8   CONTRIBUTING FACTOR VEHICLE 5  1057417 non-null  object 
 9   VEHICLE TYPE CODE 1            1057417 non-null  object 
 10  VEHICLE TYPE CODE 2            1057417 non-null  object 
 11  VEHICLE TYPE CODE 3            1057417 non-null  object 
 12  VEHICLE TYPE C

<ipython-input-4-9402aef63446>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  mvc_processed['temp_datetime'] = pd.to_datetime(mvc_processed['CRASH TIME'].astype(str))
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8146762875678538
Confusion Matrix:
 [[159760   5079      0      0]
 [ 33727  12531      0      0]
 [   179     27      0      0]
 [   101     80      0      0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.97      0.89    164839
           1       0.71      0.27      0.39     46258
           2       0.00      0.00      0.00       206
           3       0.00      0.00      0.00       181

    accuracy                           0.81    211484
   macro avg       0.38      0.31      0.32    211484
weighted avg       0.80      0.81      0.78    211484



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize the RandomForestClassifier
# Setting class_weight='balanced' to handle imbalanced dataset if needed
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced',
                            n_jobs=-1)

# Fit the RandomForest model on the training data
rf.fit(X_train_scaled, y_train)

# Predict the labels for the test set
predictions = rf.predict(X_test_scaled)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, predictions)

# Output the accuracy
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 82.44%


In [11]:
print(classification_report(y_test, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.83      0.97      0.90    164839
           1       0.74      0.31      0.44     46258
           2       1.00      0.00      0.01       206
           3       0.00      0.00      0.00       181

    accuracy                           0.82    211484
   macro avg       0.64      0.32      0.34    211484
weighted avg       0.81      0.82      0.80    211484



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, n_jobs=-1,
                            seed=42, use_label_encoder=False)

# Fit the XGBoost model on the training data
xgb_clf.fit(X_train_scaled, y_train, eval_metric='logloss')

# Predict the labels for the test set
predictions_xgb = xgb_clf.predict(X_test_scaled)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, predictions_xgb)

# Output the accuracy
print(f"Accuracy: {accuracy:.2%}")


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Accuracy: 82.60%


In [13]:
print(classification_report(y_test, predictions_xgb))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.84      0.96      0.90    164839
           1       0.73      0.34      0.46     46258
           2       0.00      0.00      0.00       206
           3       0.00      0.00      0.00       181

    accuracy                           0.83    211484
   macro avg       0.39      0.33      0.34    211484
weighted avg       0.81      0.83      0.80    211484



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Addressing class imbalance

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Fit SMOTE on the training data only
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Check the balance
print(f"Original training dataset shape {np.bincount(y_train)}")
print(f"Resampled training dataset shape {np.bincount(y_train_smote)}")


Original training dataset shape [659354 185031    824    724]
Resampled training dataset shape [659354 659354 659354 659354]


In [16]:
# Run logisitic regression model on resampled training dataset

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, n_jobs=-1)

# Fit the model on the training data
log_reg.fit(X_train_smote, y_train_smote)

# Predict the labels for both the training and test sets
train_predictions_lr = log_reg.predict(X_train_smote)
test_predictions_lr = log_reg.predict(X_test_scaled)



ValueError: Found input variables with inconsistent numbers of samples: [845933, 2637416]

In [17]:
# Generate classification reports

train_report = classification_report(y_train_smote, train_predictions_lr)
test_report = classification_report(y_test, test_predictions_lr)

# Print the classification reports
print("Training Classification Report:\n", train_report)
print("Testing Classification Report:\n", test_report)


Training Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.55      0.53    659354
           1       0.49      0.20      0.28    659354
           2       0.55      0.70      0.62    659354
           3       0.64      0.80      0.71    659354

    accuracy                           0.56   2637416
   macro avg       0.55      0.56      0.53   2637416
weighted avg       0.55      0.56      0.53   2637416

Testing Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.54      0.67    164839
           1       0.38      0.20      0.26     46258
           2       0.00      0.54      0.00       206
           3       0.00      0.80      0.01       181

    accuracy                           0.47    211484
   macro avg       0.32      0.52      0.24    211484
weighted avg       0.77      0.47      0.58    211484

